In [6]:
# keep your existing torch; just add the compressor
!uv pip install llmcompressor

# # Keep TF/Flax/JAX off (optional but tidy for LLM-only)
# export TRANSFORMERS_NO_TF=1
# export TRANSFORMERS_NO_FLAX=1
# export TRANSFORMERS_NO_JAX=1

# # 1) Remove torchvision (prevents the vision losses from loading)
# !uv pip uninstall torchvision

# # 2) Make sure these core libs are present and compatible
# !uv pip install -qU --system "transformers>=4.43,<4.46" "accelerate>=0.33" "safetensors>=0.4.4" "llmcompressor>=0.6.0"

# 3) (Optional) If you *must* keep torchvision, match it to your torch build.
#    Example for torch 2.6.0+cu124:
# uv pip install -qU --system --reinstall --no-deps "torchvision==0.21.0+cu124"

Using Python 3.11.11 environment at: /workspace/arc-agi-2025/.venv


In [2]:
model_to_compress = "Trelis/arc-1-fake-ttt-blended-c802"
output_dir = f"{model_to_compress}-fp8"

from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_to_compress, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_to_compress)

from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import QuantizationModifier

# Configure the simple PTQ quantization
recipe = QuantizationModifier(
  targets="Linear", scheme="FP8_DYNAMIC", ignore=["lm_head"])

# Apply the quantization algorithm.
oneshot(model=model, recipe=recipe)

# Save the model.
SAVE_DIR = model_to_compress.rstrip("/").split("/")[-1] + "-FP8-Dynamic"
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

from huggingface_hub import HfApi

owner = "Trelis"  # use your exact org handle/case
repo_name = "arc-1-fake-ttt-blended-c802-fp8"
repo_id = f"{owner}/{repo_name}"

api = HfApi()
api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)  # add private=True if desired

# Ensure large weights go to LFS
from pathlib import Path
(Path(output_dir)/".gitattributes").write_text(
    "*.safetensors filter=lfs diff=lfs merge=lfs -text\n*.bin filter=lfs diff=lfs merge=lfs -text\n"
)

api.upload_folder(
    repo_id=repo_id,
    folder_path=output_dir,
    repo_type="model",
    commit_message="Add FP8 (W8A8 dynamic activations) compressed checkpoint",
)
print(f"Pushed to https://huggingface.co/{repo_id}")


ModuleNotFoundError: Could not import module 'PreTrainedModel'. Are this object's requirements defined correctly?

In [ ]:
from huggingface_hub import HfApi

owner = "Trelis"  # use your exact org handle/case
repo_name = "arc-1-fake-ttt-blended-c802-fp8"
repo_id = f"{owner}/{repo_name}"

api = HfApi()
api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)  # add private=True if desired

# Ensure large weights go to LFS
from pathlib import Path
(Path(output_dir)/".gitattributes").write_text(
    "*.safetensors filter=lfs diff=lfs merge=lfs -text\n*.bin filter=lfs diff=lfs merge=lfs -text\n"
)

api.upload_folder(
    repo_id=repo_id,
    folder_path=output_dir,
    repo_type="model",
    commit_message="Add FP8 (W8A8 dynamic activations) compressed checkpoint",
)
print(f"Pushed to https://huggingface.co/{repo_id}")
